## 0. Import packages

In [25]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
from math import sqrt
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [26]:
import pickle
train_df = pickle.load(open("pickle_dumps/train_df.p", "rb"))

In [27]:
train_df

,user_id,item_id,recommend
0,1267123,[12509],[True]
1,67583,"[48, 797, 2464, 5478, 6531, 12483, 13333, 14493]","[True, True, True, True, True, True, True, True]"
2,572187,[4215],[True]
3,995160,[8009],[True]
4,1145644,[10388],[True]
...,...,...,...
1188484,441213,"[2678, 14459]","[True, True]"
1188485,502181,"[3635, 3004, 3783]","[True, True, True]"
1188486,1218503,[11693],[True]
1188487,1294191,[12319],[True]


In [28]:
data = train_df.set_index("user_id").to_dict()["item_id"]

In [29]:
def eclat(prefix, items):
    frequents = []
    while items:
        i,itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup:
            frequents.append((frozenset(prefix + [i]), isupp))
            suffix = [] 
            for j, ojtids in items:
                jtids = set(itids) & set(ojtids)
                if len(jtids) >= minsup:
                    suffix.append((j,jtids))
            frequents.extend(eclat(prefix+[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True)))
    return frequents

In [30]:
%%time

minsup = 20
# 20: 30s, 15: 1m23s
frequent_itemsets = eclat([], sorted(data.items(), key=lambda item: len(item[1]), reverse=True))
print(len(frequent_itemsets))

8071
CPU times: user 32.4 s, sys: 7.96 ms, total: 32.4 s
Wall time: 32.4 s


In [31]:
freq = sorted(frequent_itemsets, key=lambda x: 1000*len(x[0]) + x[1], reverse=True)

In [32]:
def subsets(itemset, include_empty_set=False):
    """ List all strict subsets of an itemset without the empty set or with the empty set if include_empty_set=True
        subsets({1,2,3}) --> [{1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]
    """
    s = list(itemset)
    if include_empty_set:
        return map(set, chain.from_iterable(combinations(s, r) for r in range(0, len(s) + 1)))
    return map(set, chain.from_iterable(combinations(s, r) for r in range(1, len(s))))

In [33]:
def deriveRules(itemsets, minconf):
    """ Returns all rules with conf >= minconf that can be derived from the itemsets.
        Return: list of association rules in the format: [(antecedent, consequent, supp, conf), ...]
    """
    search_items = dict(itemsets)
    rules = []
    for i, (item_set, supp) in enumerate(itemsets):
        if len(item_set) > 1:
            for subset in subsets(item_set):  # for each subset generate a rule
                antecedent = frozenset(subset)
                consequent = item_set - subset
                if len(consequent) != 0:
                    conf = supp / search_items[antecedent]
                    if conf >= minconf:
                        rules.append(Association(antecedent, consequent, conf, supp))

    return rules

rules = deriveRules(frequent_itemsets, 0)

# normalize support / confidence to better use it later

In [34]:
minc = 1
maxc = 0

mins = 9999999999999
maxs = -9999999999999
for a in rules:
    if a.c < minc:
        minc = a.c
    if a.c > maxc:
        maxc = a.c
    
    if a.s < mins:
        mins = a.s
    if a.s > maxs:
        maxs = a.s
            
for a in rules:
    a.c = (a.c - minc) / (maxc - minc)
    a.s = (a.s - mins) / (maxs - mins)
    # temp score that's just lhs + weighted average of c, s
    a.score = len(a.left) + sqrt(pow(a.c, 2) + pow(a.s, 2))

rules = sorted(rules, key = lambda x: x.score, reverse=True)
for i in rules:
    print(i)

Conf: 0.96	Supp: 26.00	 {1025, 2954, 132} => {7}
Conf: 0.95	Supp: 20.00	 {562, 20396, 23214} => {7}
Conf: 0.93	Supp: 27.00	 {1025, 3, 132} => {7}
Conf: 0.92	Supp: 34.00	 {1025, 171, 132} => {7}
Conf: 0.92	Supp: 23.00	 {2219, 14364, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 2219} => {7}
Conf: 0.92	Supp: 23.00	 {3, 2219, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 3327} => {7}
Conf: 0.92	Supp: 22.00	 {129, 2219, 1025} => {7}
Conf: 0.91	Supp: 21.00	 {1025, 23047, 7} => {132}
Conf: 0.91	Supp: 20.00	 {1025, 562, 3} => {7}
Conf: 0.91	Supp: 20.00	 {132, 8758, 7} => {1025}
Conf: 0.88	Supp: 23.00	 {1025, 17084, 132} => {7}
Conf: 0.88	Supp: 36.00	 {1025, 132, 3327} => {7}
Conf: 0.88	Supp: 21.00	 {1025, 132, 23047} => {7}
Conf: 0.88	Supp: 21.00	 {3, 132, 3327} => {7}
Conf: 0.87	Supp: 20.00	 {20396, 23214, 7} => {562}
Conf: 0.87	Supp: 20.00	 {2219, 132, 33863} => {1025}
Conf: 0.87	Supp: 26.00	 {1025, 132, 5111} => {7}
Conf: 0.85	Supp: 23.00	 {1025, 132, 14364} => {7}
Conf: 0.85	Supp: 23.00

Conf: 0.10	Supp: 28.00	 {70} => {4317}
Conf: 0.10	Supp: 28.00	 {70} => {79, 1695}
Conf: 0.07	Supp: 38.00	 {16} => {447}
Conf: 0.09	Supp: 29.00	 {5010} => {9073}
Conf: 0.10	Supp: 26.00	 {5609} => {53545}
Conf: 0.10	Supp: 27.00	 {4317} => {4093}
Conf: 0.10	Supp: 27.00	 {4317} => {5121}
Conf: 0.10	Supp: 27.00	 {4317} => {40650}
Conf: 0.10	Supp: 27.00	 {4317} => {3429}
Conf: 0.10	Supp: 27.00	 {4317} => {171, 7}
Conf: 0.09	Supp: 29.00	 {132} => {35500}
Conf: 0.09	Supp: 29.00	 {132} => {1025, 3}
Conf: 0.09	Supp: 29.00	 {132} => {23214}
Conf: 0.09	Supp: 29.00	 {132} => {129, 7}
Conf: 0.10	Supp: 21.00	 {9833} => {17878}
Conf: 0.10	Supp: 21.00	 {9833} => {14168}
Conf: 0.10	Supp: 21.00	 {9833} => {20396}
Conf: 0.10	Supp: 21.00	 {9833} => {171, 35509}
Conf: 0.10	Supp: 21.00	 {9833} => {5150}
Conf: 0.10	Supp: 21.00	 {9833} => {5366}
Conf: 0.10	Supp: 21.00	 {9833} => {2880}
Conf: 0.10	Supp: 21.00	 {9833} => {171, 1334}
Conf: 0.10	Supp: 21.00	 {9833} => {562, 171}
Conf: 0.10	Supp: 21.00	 {9833} => {

Conf: 0.02	Supp: 23.00	 {7} => {3, 171}
Conf: 0.02	Supp: 23.00	 {7} => {1025, 3, 3327}
Conf: 0.02	Supp: 23.00	 {7} => {10808}
Conf: 0.02	Supp: 23.00	 {7} => {132, 22767}
Conf: 0.02	Supp: 23.00	 {7} => {1025, 22767}
Conf: 0.02	Supp: 23.00	 {7} => {562, 14284}
Conf: 0.02	Supp: 23.00	 {7} => {90955, 132}
Conf: 0.02	Supp: 23.00	 {7} => {90955, 171}
Conf: 0.02	Supp: 23.00	 {7} => {10484}
Conf: 0.02	Supp: 23.00	 {7} => {4705, 3327}
Conf: 0.02	Supp: 23.00	 {7} => {1025, 8597}
Conf: 0.02	Supp: 23.00	 {7} => {6270, 3327}
Conf: 0.02	Supp: 23.00	 {7} => {9833, 171}
Conf: 0.02	Supp: 23.00	 {7} => {1025, 562, 132}
Conf: 0.02	Supp: 23.00	 {7} => {3897, 562}
Conf: 0.02	Supp: 23.00	 {7} => {2954, 70}
Conf: 0.02	Supp: 23.00	 {7} => {5010, 171}
Conf: 0.03	Supp: 20.00	 {171} => {167545}
Conf: 0.03	Supp: 20.00	 {171} => {36219}
Conf: 0.03	Supp: 20.00	 {171} => {134899}
Conf: 0.03	Supp: 20.00	 {171} => {53120}
Conf: 0.03	Supp: 20.00	 {171} => {25324}
Conf: 0.03	Supp: 20.00	 {171} => {23577}
Conf: 0.03	Supp

In [35]:
len(rules)

7566

# ??. Store the association rules as pickle file for further use

In [36]:
import pickle
pickle.dump(rules, open("pickle_dumps/article_rules.p", "wb"))